# Loading assets

In [102]:

import warnings
warnings.filterwarnings("ignore")

from sklearn import dummy #Majority classifier
from sklearn.linear_model import LogisticRegression #will be used to build custom classifier
from sklearn.naive_bayes import GaussianNB #will be used to build custom classifier
from sklearn.ensemble import RandomForestClassifier #will be used to build custom classifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import Imputer
from sklearn import preprocessing




import numpy as np
import pandas as pd
import csv
import operator


from sklearn import cross_validation  #No train-test set splitting will be done, I'll use the library
from sklearn import datasets

input_file = "/tmp/train.csv"
train = pd.read_csv(input_file, header = 0)
input_file = "/tmp/test.csv"
test = pd.read_csv(input_file, header = 0)

df=pd.concat([train,test])

cols_to_convert = ['job','marital','education','default','housing','loan','contact','month','day_of_week','campaign','pdays','previous','poutcome']
le = preprocessing.LabelEncoder()
for col in cols_to_convert:
    df[col]=le.fit_transform(df[col])
    
test = df.tail(1120)
train = df.head(df.shape[0]-1120)

test = test.ix[:,0:-1]  #ycol added


train_X=train.ix[:,0:-1]
train_Y=train.ix[:,-1]


### We turn the data in a (samples, feature) matrix:

In [57]:
#train = Imputer().fit_transform(train)
n_samples = len(train_Y)
#data = train_X.reshape((n_samples, -1))

# For the majority class classifier, I use sklearn.dummy.DummyClassifier.
### Custom classifier buidling will be done later

In [58]:
classifier = dummy.DummyClassifier(strategy='most_frequent')

In [59]:
for key, label in zip([classifier], ['Majority class classifier']):
    scores = cross_validation.cross_val_score(key, train_X, train_Y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.89 (+/- 0.00) [Majority class classifier]


### Very poor accuracy. It gets it right 10% of the time. No big deal, since it's a digit from [0,9]. They all have 0.1 probability of appearance. This is also a balanced dataset. No sampling bias is existent

In [40]:
expected = digits.target[n_samples / 2:]
predicted = classifier.predict(data[n_samples / 2:])

# Custom ensemble classifier building
#### weights of the bagged classifiers are equal, it would be a good idea to tune them otherwise

In [141]:
'''class EnsembleClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, keys):
        self.keys = keys
        
    def fit(self, X, y):
        
        for key in self.keys:
            key.fit(X, y)

    def predict(self, X):
        
        self.classes_ = np.asarray([key.predict(X) for key in self.keys])
        maj = np.asarray([np.argmax(np.bincount(self.classes_[:,c])) for c in range(self.classes_.shape[1])])

        return maj

    def predict_proba(self, X):
        
        self.probas_ = [key.predict_proba(X) for key in self.keys]
        avg = np.average(self.probas_)

        return avg'''
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = RandomForestClassifier(n_estimators=1500)
estimators.append(('rf', model2))
model3 = SVC()
#estimators.append(('svm', model3))
model4 = GradientBoostingClassifier(n_estimators=1500)
#estimators.append(('XGBoost', model4))
# create the ensemble model
ensemble = VotingClassifier(estimators)
#results = cross_validation.cross_val_score(ensemble, X, Y, cv=kfold)

In [85]:
np.random.seed(1327)
key1 = LogisticRegression()
key2 = RandomForestClassifier(n_estimators=1500)
key3 = GradientBoostingClassifier(n_estimators=1500)
key0 = SVC()

for key, label in zip([key1, key2, key3, key0, ensemble], ['Logistic Regression', 'Random Forest', 'XGBoost', 'SVM' ,'ensemble']):
    scores = cross_validation.cross_val_score(key, train_X, train_Y, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))



Accuracy: 0.91 (+/- 0.01) [Logistic Regression]
Accuracy: 0.91 (+/- 0.01) [Random Forest]
Accuracy: 0.90 (+/- 0.02) [XGBoost]
Accuracy: 0.89 (+/- 0.00) [SVM]
Accuracy: 0.91 (+/- 0.01) [ensemble]


In [142]:
ensemble.fit(train_X,train_Y)
ensemble.estimators

[('logistic',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False)),
 ('rf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=1500, n_jobs=1, oob_score=False,
              random_state=None, verbose=0, warm_start=False))]

In [143]:
prediction = ensemble.predict(test)
prediction.tofile("/tmp/final2.csv",sep = ",")

# THE END